<a href="https://colab.research.google.com/github/cseybold/Weather/blob/main/Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install meteostat

In [ ]:
import ee
from google.colab import auth
auth.authenticate_user(project_id='weatherproj-427900')

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='weatherproj-427900')

In [ ]:
# Import Meteostat library and dependencies
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from meteostat import Point, Daily
import pandas as pd
import xarray as xr

firedf = pd.read_csv('/content/drive/MyDrive/WFIGS_inc_loc.csv')


In [ ]:
firedf = firedf[['SourceOID', 'IncidentSize', 'FireDiscoveryDateTime', 'IncidentTypeCategory', 'InitialLatitude', 'InitialLongitude', 'POOState']]
cafire = firedf[firedf['POOState'] == 'US-CA']
cafire = cafire[cafire['IncidentTypeCategory'] == 'WF']
cafire = cafire[cafire['IncidentSize'] > 1]
cafire.tail()

,SourceOID,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState
309485,32987182,1.80,6/25/2024 9:15:15 PM,WF,35.561729,-120.516991,US-CA
309497,32987205,1.25,6/25/2024 6:04:39 PM,WF,41.759312,-122.490208,US-CA
309520,32987244,11.03,6/25/2024 11:30:19 PM,WF,38.845861,-123.019703,US-CA
309569,32987303,12.00,6/26/2024 2:28:04 AM,WF,36.631882,-119.322600,US-CA
309592,32987334,53.91,6/26/2024 5:00:46 AM,WF,37.732422,-120.596351,US-CA


In [ ]:
cafire['FireDiscoveryDateTime'] = pd.to_datetime(cafire['FireDiscoveryDateTime'])
num_NAs = cafire['FireDiscoveryDateTime'].isna().sum()
print(num_NAs)
cafire.dropna(subset=['InitialLatitude'], inplace=True)
cafire = cafire.reset_index(drop=True)
cafire.tail()

0


,SourceOID,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState
3811,32987182,1.80,2024-06-25 21:15:15,WF,35.561729,-120.516991,US-CA
3812,32987205,1.25,2024-06-25 18:04:39,WF,41.759312,-122.490208,US-CA
3813,32987244,11.03,2024-06-25 23:30:19,WF,38.845861,-123.019703,US-CA
3814,32987303,12.00,2024-06-26 02:28:04,WF,36.631882,-119.322600,US-CA
3815,32987334,53.91,2024-06-26 05:00:46,WF,37.732422,-120.596351,US-CA


In [ ]:
testdf = cafire.head(50)

def weather_stats(lat, lon, time):
    # Create a Point for the location
    location = Point(lat, lon)

    # Get hourly data
    data = Daily(location, time, time + timedelta(days=1))
    data = data.fetch()

    # Check if data is empty
    if data.empty:
        return None

    # Return the temperature, precipitation, and wind speed
    return [data.iloc[0]['tavg'], data.iloc[0]['prcp'], data.iloc[0]['wspd']]

# Apply the function to each row and handle None values
def apply_weather_stats(row, i):
    stats = weather_stats(row['InitialLatitude'], row['InitialLongitude'], row['FireDiscoveryDateTime'])
    return stats[i] if stats is not None else None

testdf['Temperature'] = testdf.apply(lambda row: apply_weather_stats(row, 0), axis=1)
testdf['Precipitation'] = testdf.apply(lambda row: apply_weather_stats(row, 1), axis=1)
testdf['WindSpeed'] = testdf.apply(lambda row: apply_weather_stats(row, 2), axis=1)

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
testdf['Precipitation'].fillna(0, inplace=True)
testdf.dropna(subset=['Temperature'], inplace=True)
testdf.dropna(subset=['WindSpeed'], inplace=True)

testdf.tail()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,SourceOID,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState,Temperature,Precipitation,WindSpeed
41,23831225,4.00,2022-04-23 14:30:08,WF,40.665000,-122.636200,US-CA,12.1,0.0,4.9
42,7030775,2.85,2019-10-07 08:12:00,WF,33.773090,-116.545600,US-CA,29.3,0.0,10.3
45,23840009,80.00,2022-06-21 16:48:36,WF,33.546326,-116.043868,US-CA,32.9,0.0,12.4
46,4581975,2.00,2018-06-16 16:30:00,WF,34.665230,-118.761900,US-CA,9.3,0.0,25.6
48,7064375,350.00,2019-10-10 19:56:55,WF,33.984818,-117.217944,US-CA,19.9,0.0,13.3


In [ ]:
def get_vegetation_index(longitude, latitude, date):
    # Define the dataset
    dataset = ee.ImageCollection('NOAA/VIIRS/001/VNP13A1')

    # Filter the dataset by date and location
    filtered_dataset = dataset.filterDate(date, date + timedelta(days = 40)).filterBounds(ee.Geometry.Point([longitude, latitude]))

    # Select 'EVI' and calculate the mean
    mean_evi = filtered_dataset.select('EVI2').mean()

    # Define a region of interest
    roi = ee.Geometry.Rectangle([longitude-0.01, latitude-0.01, longitude+0.01, latitude+0.01])

    # Reduce the image over the region of interest
    mean_evi = mean_evi.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30)

    # Check if 'EVI2' exists in the dictionary
    if 'EVI2' in mean_evi.getInfo():
        mean_evi_value = mean_evi.get('EVI2').getInfo()
    else:
        mean_evi_value = None

    return mean_evi_value

testdf['VegetationIndex'] = testdf.apply(lambda row: get_vegetation_index(row['InitialLongitude'], row['InitialLatitude'], row['FireDiscoveryDateTime']), axis=1)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
def get_mfri(longitude, latitude):
    # Define the dataset
    dataset = ee.ImageCollection('LANDFIRE/Fire/MFRI/v1_2_0')

    # Filter the dataset by location
    filtered_dataset = dataset.filterBounds(ee.Geometry.Point([longitude, latitude]))

    # Check if the image collection is empty
    if filtered_dataset.size().getInfo() == 0:
        return None

    # Select 'MFRI' and calculate the mean
    mean_mfri = filtered_dataset.select('MFRI').mean()

    # Define a region of interest
    roi = ee.Geometry.Rectangle([longitude-0.01, latitude-0.01, longitude+0.01, latitude+0.01])

    # Reduce the image over the region of interest
    mean_mfri = mean_mfri.reduceRegion(reducer=ee.Reducer.mean(), geometry=roi, scale=30)

    # Get the mean MFRI value
    if 'MFRI' in mean_mfri.getInfo():
        mean_mfri_value = mean_mfri.get('MFRI').getInfo()
    else:
        mean_mfri_value = None

    return mean_mfri_value

testdf['MFRI'] = testdf.apply(lambda row: get_mfri(row['InitialLongitude'], row['InitialLatitude']), axis=1)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
testdf.dropna(subset=['VegetationIndex'], inplace=True)
testdf.dropna(subset=['MFRI'], inplace=True)
testdf = testdf.where(testdf['MFRI'] > 0)
testdf = testdf.reset_index(drop=True)

testdf.tail()

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,SourceOID,IncidentSize,FireDiscoveryDateTime,IncidentTypeCategory,InitialLatitude,InitialLongitude,POOState,Temperature,Precipitation,WindSpeed,VegetationIndex,MFRI
22,23831225.0,4.00,2022-04-23 14:30:08,WF,40.665000,-122.636200,US-CA,12.1,0.0,4.9,3302.670592,7.191576
23,7030775.0,2.85,2019-10-07 08:12:00,WF,33.773090,-116.545600,US-CA,29.3,0.0,10.3,1652.521132,35.254068
24,23840009.0,80.00,2022-06-21 16:48:36,WF,33.546326,-116.043868,US-CA,32.9,0.0,12.4,1651.378559,71.748126
25,4581975.0,2.00,2018-06-16 16:30:00,WF,34.665230,-118.761900,US-CA,9.3,0.0,25.6,1480.511875,12.864382
26,7064375.0,350.00,2019-10-10 19:56:55,WF,33.984818,-117.217944,US-CA,19.9,0.0,13.3,1459.243946,16.378446


In [ ]:
mtemp = testdf['Temperature'].mean()
mprec = testdf['Precipitation'].mean()
mwind = testdf['WindSpeed'].mean()
mveg = testdf['VegetationIndex'].mean()
mMFRI = testdf['MFRI'].mean()

sdtemp = testdf['Temperature'].std()
sdprec = testdf['Precipitation'].std()
sdwind = testdf['WindSpeed'].std()
sdveg = testdf['VegetationIndex'].std()
sdMFRI = testdf['MFRI'].std()

testdata = pd.DataFrame({
    'Temperature': [mtemp, sdtemp],
    'Precipitation': [mprec, sdprec],
    'WindSpeed': [mwind, sdwind],
    'VegetationIndex': [mveg, sdveg],
    'MFRI': [mMFRI, sdMFRI]
}, index=['mean', 'std'])

testdata.head()

,Temperature,Precipitation,WindSpeed,VegetationIndex,MFRI
mean,21.58750,0.0,12.841667,1807.864464,23.072327
std,6.39472,0.0,6.135493,463.556331,19.258073


In [ ]:
# Define the data for each column
testdata2 = pd.DataFrame({
    'Temperature': [None],
    'Precipitation': [None],
    'WindSpeed': [None],
    'VegetationIndex': [None],
    'MFRI': [None]
})

testdata2['Latitude'] = float(input("Enter latitude: "))
testdata2['Longitude'] = float(input("Enter longitude: "))
testdata2['Date'] = pd.to_datetime(input("Enter date (YYYY-MM-DD): "))

# Create the DataFrame
testdata2 = pd.DataFrame(testdata2, index=[0])

# Apply the function to each row and handle None values
def apply_weather_stats2(row, i):
    stats = weather_stats(row['Latitude'], row['Longitude'], row['Date'])
    return stats[i] if stats is not None else None

new_row = testdata2.apply(lambda row: [apply_weather_stats2(row, 0),
                                        apply_weather_stats2(row, 1),
                                        apply_weather_stats2(row, 2),
                                        get_vegetation_index(row['Longitude'], row['Latitude'], row['Date']),
                                        get_mfri(row['Longitude'], row['Latitude'])], axis=1)[0]

testdata.loc['new_data'] = new_row # Add the new row with index 'new_data'

testdata.head()

In [ ]:
# Create an empty Series to store the results
result = pd.Series(index=testdata.columns)

# Iterate over each column in the DataFrame
for column in testdata.columns:
    if testdata.loc['std', column] == 0:
        value = 0
    else:
        value = (testdata.loc['new_data', column] - testdata.loc['mean', column]) / testdata.loc['std', column]

    # Add the result to the new DataFrame
    result.loc[column] = value

# Print the result
testdata.loc['stdaway'] = result

testdata.head()

In [ ]:
fire_danger = 2*testdata.loc['stdaway', 'Temperature'] - testdata.loc['stdaway', 'Precipitation'] + testdata.loc['stdaway', 'WindSpeed'] + testdata.loc['stdaway', 'VegetationIndex'] - testdata.loc['stdaway', 'MFRI']/3
print(fire_danger)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Assume df is your DataFrame and "wildfire_size" is the target variable
X = testdf[['Temperature', 'Precipitation', 'WindSpeed', 'VegetationIndex', 'MFRI']]
y = testdf["IncidentSize"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2003)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model.add(Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)))  # L2 regularization
model.add(Dense(1, activation='linear'))  # Output layer for regression

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the new data for a few epochs
model.fit(X_train, y_train, epochs=50)  # Adjust the number of epochs as needed
